In [ ]:
!pip install torch
!pip install transformers
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import pandas as pd
# Define the QAModel class for question-answering
class QAModel(nn.Module):
    def __init__(self, bert_model):
        super(QAModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, 2)
    def forward(self, input_ids, attention_mask):
        # Forward pass through the model
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits
def load_dataset(dataset_path):
    # Load dataset from the provided path
    dataset = pd.read_csv(dataset_path)
    return dataset
def get_ted_talk_details(author, dataset):
    # Filter dataset based on the author and return talks as a list of dictionaries
    filtered_data = dataset[dataset['author'] == author]
    talks = filtered_data.to_dict('records')
    return talks
def ask_question(question, context, tokenizer, model):
    # Tokenize the question and context, encode them, and perform inference
    input_ids = tokenizer.encode(question, context, truncation=True, max_length=512)
    attention_mask = [1] * len(input_ids)
    input_ids = torch.tensor([input_ids])
    attention_mask = torch.tensor([attention_mask])
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        start_scores, end_scores = logits.split(1, dim=-1)
        start_scores = start_scores.squeeze(-1)
        end_scores = end_scores.squeeze(-1)
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    answer = tokenizer.decode(input_ids[0, start_index:end_index + 1])
    return answer
def main():
    dataset_path = "/content/data.csv"
    dataset = load_dataset(dataset_path)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased')
    model = QAModel(bert_model)
    author = input("Enter the name of the author: ")
    talks = get_ted_talk_details(author, dataset)
    if not talks:
        print("No talks found for the specified author.")
        return
    while True:
        question = input("Ask a question (or enter 'q' to quit): ")
        if question == 'q':
            break
        for talk in talks:
            title = talk['title']
            date = talk['date']
            views = talk['views']
            likes = talk['likes']
            link = talk['link']
            context = f"Title: {title}. Date: {date}."
            answer = ask_question(question, context, tokenizer, model)
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f"Title: {title}")
            print(f"Date: {date}")
            print(f"Views: {views}")
            print(f"Likes: {likes}")
            print(f"Link: {link}")
            print()
if __name__ == "__main__":
    main()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Enter the name of the author: Zac Goldsmith
Ask a question (or enter 'q' to quit): give me details
Question: give me details
Answer: [CLS]
Title: How to boost nature-based solutions to climate change
Date: Oct-21
Views: 30000
Likes: 913
Link: https://ted.com/talks/zac_goldsmith_how_to_boost_nature_based_solutions_to_climate_change

Ask a question (or enter 'q' to quit): q


In [ ]:
!pip install telebot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=50fa4b637e46e20244a7c8438958a6f81fb220c8bf024bfe0118138070f4c99b
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pyTelegramBotAPI


In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import pandas as pd
import telebot
from telebot import types
class QAModel(nn.Module):
    def __init__(self, bert_model):
        super(QAModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, 2)
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits
def load_dataset(dataset_path):
    dataset = pd.read_csv(dataset_path)
    return dataset
def get_ted_talk_details(author, dataset):
    filtered_data = dataset[dataset['author'] == author]
    talks = filtered_data.to_dict('records')
    return talks
def ask_question(question, context, tokenizer, model):
    input_ids = tokenizer.encode(question, context, truncation=True, max_length=512)
    attention_mask = [1] * len(input_ids)
    input_ids = torch.tensor([input_ids])
    attention_mask = torch.tensor([attention_mask])
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        start_scores, end_scores = logits.split(1, dim=-1)
        start_scores = start_scores.squeeze(-1)
        end_scores = end_scores.squeeze(-1)
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    answer = tokenizer.decode(input_ids[0, start_index:end_index + 1])
    return answer
def handle_messages(message):
    dataset_path = "/content/data.csv"
    dataset = load_dataset(dataset_path)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased')
    model = QAModel(bert_model)
    author = message.text
    talks = get_ted_talk_details(author, dataset)
    if not talks:
        bot.reply_to(message, "No talks found for the specified author.")
        return
    bot.reply_to(message, "Author: " + author)
    for talk in talks:
        title = talk['title']
        date = talk['date']
        views = talk['views']
        likes = talk['likes']
        link = talk['link']
        context = f"Title: {title}. Date: {date}."
        question = bot.reply_to(message, "Ask a question (or enter 'q' to quit): ")
        bot.register_next_step_handler(question, process_question, context=context, tokenizer=tokenizer, model=model,
                                       title=title, date=date, views=views, likes=likes, link=link)
def process_question(message, context, tokenizer, model, title, date, views, likes, link):
    question = message.text
    if question.lower() == 'q':
        bot.reply_to(message, "Thanks for using the bot. Have a great day!")
        return
    answer = ask_question(question, context, tokenizer, model)
    response = f"Question: {question}\n" \
               f"Answer: {answer}\n" \
               f"Title: {title}\n" \
               f"Date: {date}\n" \
               f"Views: {views}\n" \
               f"Likes: {likes}\n" \
               f"Link: {link}"
    bot.reply_to(message, response)
bot = telebot.TeleBot("your bot id")
@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message, "Welcome to the TED Talk QA Bot!\nPlease enter the name of the author:")
@bot.message_handler(func=lambda message: True)
def handle_all_messages(message):
    handle_messages(message)
bot.polling()